In [ ]:
!ls

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pickle

In [ ]:
plt.rcParams["figure.figsize"] = [16, 9]
# plt.rcParams["figure.dpi"] = 300
plt.rcParams["font.size"] = 20
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["axes.titlesize"] = 24
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.rcParams["font.family"] = "serif"

In [ ]:
FONT_SIZE_TITLE_PLOT = 48  # 40
FONT_SIZE_TITLE_AX = 36  # 30
FONT_SIZE_LABEL = 30  # 24
FONT_SIZE_TICKS = 24  # 20
FONT_SIZE_LEGEND = 32  # 28

In [ ]:
PROJECT_FOLDER = "PycharmProjects/thesis-gan"

In [ ]:
stock_names = ["KO", "PEP", "NVDA"]
n_stocks = len(stock_names)
RUN_ID_PRICE = "oy1upczq"
SIGMA_SCALERs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
SEEDS = range(14, 15)

In [ ]:
stock_names = ["KO", "PEP"]
n_stocks = len(stock_names)
RUN_ID_PRICE = "19oly5ng"
SIGMA_SCALERs = [0.4]
SEEDS = range(10, 11)

In [ ]:
seed2priceWoPer = dict()
for seed in SEEDS:
    PATH_PICKLE_PRICE_NOPER = f"{PROJECT_FOLDER}/storage/thesis-gan/{RUN_ID_PRICE}/perturbations/shock/perturbation_seed={seed}_sigmascaler=None.pickle"

    with open(PATH_PICKLE_PRICE_NOPER, "rb") as handle:
        dict_no_per = pickle.load(handle)

    prices_no_per = dict_no_per["pred_prices"].numpy()
    seed2priceWoPer[seed] = prices_no_per
seed2priceWoPer[SEEDS[0]].shape

In [ ]:
PATH_PICKLE_PRICE_REALS = f"{PROJECT_FOLDER}/storage/thesis-gan/{RUN_ID_PRICE}/reals.pickle"
with open(PATH_PICKLE_PRICE_REALS, "rb") as handle:
    dict_reals = pickle.load(handle)
prices_reals = dict_reals["prices"][:, : seed2priceWoPer[SEEDS[0]].shape[-1]]
prices_reals.shape

In [ ]:
seed2priceWPerS = dict()
for seed in SEEDS:
    PATHs_PICKLE_PRICE_PER = [
        f"{PROJECT_FOLDER}/storage/thesis-gan/{RUN_ID_PRICE}/perturbations/shock/perturbation_seed={seed}_sigmascaler={sigma_scaler}.pickle"
        for sigma_scaler in SIGMA_SCALERs
    ]

    l = list()
    for path in PATHs_PICKLE_PRICE_PER:
        with open(path, "rb") as handle:
            dict_per = pickle.load(handle)
        prices_per = dict_per["pred_prices"].numpy()
        l.append(prices_per)

    prices_per = np.asarray(l).transpose(1, 0, 2)
    seed2priceWPerS[seed] = prices_per
seed2priceWPerS[SEEDS[0]].shape

In [ ]:
with open(
    f"{PROJECT_FOLDER}/storage/thesis-gan/{RUN_ID_PRICE}/perturbations/shock/perturbation_seed={seed}_sigmascaler=0.4_updown.pickle",
    "rb",
) as f:
    d = pickle.load(f)
    pricesWPer_updown = d["pred_prices"].numpy()
with open(
    f"{PROJECT_FOLDER}/storage/thesis-gan/{RUN_ID_PRICE}/perturbations/shock/perturbation_seed={seed}_sigmascaler=0.4_downup.pickle",
    "rb",
) as f:
    d = pickle.load(f)
    pricesWPer_downup = d["pred_prices"].numpy()
pricesWPer_updown.shape, pricesWPer_downup.shape

In [ ]:
history_indexes = np.arange(390)
continuation_indexes = np.arange(390, prices_reals.shape[-1])
history_indexes.shape, continuation_indexes.shape

In [ ]:
decoder_length = 50

In [ ]:
start_iteration_perturbation = 8
n_iterations = 17

start_perturbation_n1 = 390 + decoder_length * start_iteration_perturbation + 1
start_perturbation_n2 = start_perturbation_n1 + decoder_length * n_iterations
end_perturbation = start_perturbation_n2 + decoder_length * n_iterations
start_perturbation_n1, start_perturbation_n2, end_perturbation

In [ ]:
pricesWoPer = seed2priceWoPer[SEEDS[0]]
prices_reals.shape, pricesWoPer.shape, pricesWPer_updown.shape, pricesWPer_updown.shape

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 9))
axes = axes.ravel()

for ax, stock_name, price_real, priceWoPer, priceWPer_updown, priceWPer_downup in zip(
    axes, stock_names, prices_reals, pricesWoPer, pricesWPer_updown, pricesWPer_downup
):
    ax.plot(history_indexes, priceWoPer[:390], color="C0")
    ax.axvline(x=390, color="r")

    # ax.plot(continuation_indexes, price_real[390:], color="C1", label="Real" if add_label else None)

    ax.plot(continuation_indexes, priceWoPer[390:], color="C2", label="W/o Per" if stock_name == "KO" else None)
    ax.plot(
        continuation_indexes, priceWPer_updown[390:], color="C3", label="W/ Per UpDown" if stock_name == "KO" else None
    )
    # ax.plot(continuation_indexes, priceWPer_downup[390:], color="C4", label="W/ Per DownUp" if stock_name == 'KO' else None)

    min_ = min(priceWoPer.min(), priceWPer_updown.min(), priceWPer_downup.min())
    max_ = max(priceWoPer.max(), priceWPer_updown.max(), priceWPer_downup.max())
    if stock_name == "KO":
        ax.add_patch(
            Rectangle(
                xy=(start_perturbation_n1, min_),
                width=start_perturbation_n2 - start_perturbation_n1,
                height=max_ - min_,
                color="r",
                alpha=0.3,
            )
        )
    ax.add_patch(
        Rectangle(
            xy=(start_perturbation_n2 + 1, min_),
            width=start_perturbation_n2 + 1 - start_perturbation_n1,
            height=max_ - min_,
            color="g",
            alpha=0.3,
        )
    )

    ax.set_title(stock_name, fontsize=FONT_SIZE_TITLE_AX)
    ax.set_xlabel("Steps", fontsize=FONT_SIZE_LABEL)
    ax.set_ylabel("Price ($)", fontsize=FONT_SIZE_LABEL, rotation=90)
    ax.xaxis.set_tick_params(labelsize=12)
    ax.yaxis.set_tick_params(labelsize=FONT_SIZE_TICKS)
    ax.set_xticks([0, 390, start_perturbation_n2, priceWoPer.shape[-1]])

fig.suptitle("Introducing perturbations", fontsize=FONT_SIZE_TITLE_PLOT, y=1.03)
fig.legend(loc="upper center", ncol=3, fontsize=FONT_SIZE_LEGEND, frameon=False, bbox_to_anchor=(0.5, 0.97))
fig.tight_layout()
# plt.savefig(f"{PROJECT_FOLDER}/plot_finali/multistock/perturbations/perturbed_NVDA_one.pdf")
plt.show()
plt.close(fig)

In [ ]:
pricesWPer_updown

In [ ]:
np.corrcoef(pricesWPer_updown[:, end_perturbation:])

# PEP & KO

In [ ]:
corr_real = np.corrcoef(prices_reals)[0, 1]
corr_real = round(corr_real, 2)

In [ ]:
seed2corr_KOwoPer_PEPwoPer = dict()
for seed, priceWoPer in seed2priceWoPer.items():
    KOwoPer, PEPwoPer = priceWoPer[:, start_perturbation_n2 + 150 :]
    corr_KOwoPer_PEPwoPer = np.corrcoef(KOwoPer, PEPwoPer)[0, 1]
    corr_KOwoPer_PEPwoPer = round(corr_KOwoPer_PEPwoPer, 2)
    if corr_KOwoPer_PEPwoPer >= 0.7:
        seed2corr_KOwoPer_PEPwoPer[seed] = corr_KOwoPer_PEPwoPer

In [ ]:
seed2sigma2corr_KOwPer_PEPwoPer, seed2sigma2corr_KOwPer_PEPwPer = dict(), dict()
for seed in seed2corr_KOwoPer_PEPwoPer:
    _, PEPwoPer = seed2priceWoPer[seed][:, start_perturbation_n2 + 150 :]
    KOwPerS, PEPwPerS = seed2priceWPerS[seed][:, :, start_perturbation_n2 + 150 :]

    seed2sigma2corr_KOwPer_PEPwoPer[seed] = dict()
    seed2sigma2corr_KOwPer_PEPwPer[seed] = dict()
    for sigma_scaler, KOwPer, PEPwPer in zip(SIGMA_SCALERs, KOwPerS, PEPwPerS):
        seed2sigma2corr_KOwPer_PEPwoPer[seed][sigma_scaler] = round(np.corrcoef(KOwPer, PEPwoPer)[0, 1], 2)
        seed2sigma2corr_KOwPer_PEPwPer[seed][sigma_scaler] = round(np.corrcoef(KOwPer, PEPwPer)[0, 1], 2)

In [ ]:
print("\t\t\t\t\tKOwoPer_PEPwoPer\tKOwPer_PEPwoPer\tKOwPer_PEPwPer")

for seed in seed2corr_KOwoPer_PEPwoPer:
    corr_KOwoPer_PEPwoPer = seed2corr_KOwoPer_PEPwoPer[seed]
    for sigma in SIGMA_SCALERs:
        sigma2corr_KOwPer_PEPwoPer = seed2sigma2corr_KOwPer_PEPwoPer[seed][sigma]
        sigma2corr_KOwPer_PEPwPer = seed2sigma2corr_KOwPer_PEPwPer[seed][sigma]
        print(
            f"Seed: {seed} - Sigma: {sigma}\t\t{corr_KOwoPer_PEPwoPer}\t\t\t{sigma2corr_KOwPer_PEPwoPer}\t\t\t{sigma2corr_KOwPer_PEPwPer}"
        )
    print()

In [ ]:
good_seeds = list(seed2corr_KOwoPer_PEPwoPer.keys())

In [ ]:
l = list()
for d in seed2sigma2corr_KOwPer_PEPwoPer.values():
    l.append(list(d.values()))
corrs_KOwPer_PEPwoPer = np.asarray(l)
corrs_KOwPer_PEPwoPer_mean = np.mean(corrs_KOwPer_PEPwoPer, axis=0)
corrs_KOwPer_PEPwoPer_std = np.std(corrs_KOwPer_PEPwoPer, axis=0) / np.sqrt(len(corrs_KOwPer_PEPwoPer))
corrs_KOwPer_PEPwoPer_mean.shape, corrs_KOwPer_PEPwoPer_std.shape

In [ ]:
l = list()
for d in seed2sigma2corr_KOwPer_PEPwPer.values():
    l.append(list(d.values()))
corrs_KOwPer_PEPwPer = np.asarray(l)
corrs_KOwPer_PEPwPer_mean = np.mean(corrs_KOwPer_PEPwPer, axis=0)
corrs_KOwPer_PEPwPer_std = np.std(corrs_KOwPer_PEPwPer, axis=0) / np.sqrt(len(corrs_KOwPer_PEPwPer))
corrs_KOwPer_PEPwPer_mean.shape, corrs_KOwPer_PEPwPer_std.shape

In [ ]:
fig = plt.figure(figsize=(16, 9))

plt.plot(SIGMA_SCALERs, [corr_real] * len(SIGMA_SCALERs), label=r"$\rho(KO, PEP)$", color="C1")
plt.plot(
    SIGMA_SCALERs,
    [seed2corr_KOwoPer_PEPwoPer[seed]] * len(SIGMA_SCALERs),
    label=r"$\rho(\widehat{KO}, \widehat{PEP})$",
    color="C2",
)

plt.fill_between(
    SIGMA_SCALERs,
    corrs_KOwPer_PEPwoPer_mean,
    corrs_KOwPer_PEPwoPer_mean + corrs_KOwPer_PEPwoPer_std,
    color="C3",
    alpha=0.2,
)
plt.fill_between(
    SIGMA_SCALERs,
    corrs_KOwPer_PEPwoPer_mean,
    corrs_KOwPer_PEPwoPer_mean - corrs_KOwPer_PEPwoPer_std,
    color="C3",
    alpha=0.2,
)
plt.plot(SIGMA_SCALERs, corrs_KOwPer_PEPwoPer_mean, label=r"$\rho(\widehat{KO_p}, \widehat{PEP})$", color="C3")

plt.fill_between(
    SIGMA_SCALERs,
    corrs_KOwPer_PEPwPer_mean,
    corrs_KOwPer_PEPwPer_mean + corrs_KOwPer_PEPwPer_std,
    color="C4",
    alpha=0.2,
)
plt.fill_between(
    SIGMA_SCALERs,
    corrs_KOwPer_PEPwPer_mean,
    corrs_KOwPer_PEPwPer_mean - corrs_KOwPer_PEPwPer_std,
    color="C4",
    alpha=0.2,
)
plt.plot(SIGMA_SCALERs, corrs_KOwPer_PEPwPer_mean, color="C4", label=r"$\rho(\widehat{KO_p}, \widehat{PEP_m})$")

plt.ylim((-1, 1))
plt.xlabel(r"$\alpha$", fontdict={"fontsize": FONT_SIZE_LABEL})
plt.ylabel("Correlation\ncoefficient", fontdict={"fontsize": FONT_SIZE_LABEL})
plt.title(
    "Reactive correlation dynamics",
    fontsize=FONT_SIZE_TITLE_PLOT,
)
plt.legend(
    loc="best",
    ncol=2,
    fontsize=FONT_SIZE_LEGEND,
    frameon=False,
    # bbox_to_anchor=(0.5, 1.)
)
plt.tight_layout()
# plt.savefig(f"{PROJECT_FOLDER}/plot_finali/multistock/perturbations/reactive_shock.pdf")
plt.show()
plt.close(fig)

In [ ]:
seed = good_seeds[0]

priceWoPer = seed2priceWoPer[seed]
priceWPer = seed2priceWPerS[seed]

prices_reals.shape, priceWoPer.shape, priceWPer.shape

In [ ]:
price

# PEP & KO & NVDA

In [ ]:
stock_names, prices_reals.shape, seed2priceWoPer[SEEDS[0]].shape, seed2priceWPerS[SEEDS[0]].shape

In [ ]:
priceWoPer = seed2priceWoPer[SEEDS[0]]
priceWPer = seed2priceWPerS[SEEDS[0]][:, :]
priceWoPer.shape, priceWPer.shape

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 9))
axes = axes.ravel()

add_label = True
add_label_ = True
for (
    ax,
    stock_name,
    price_real,
    price_no_per,
    price_per,
) in zip(axes, stock_names, prices_reals, priceWoPer, priceWPer):
    ax.plot(history_indexes, price_no_per[:390], color="C0")
    # ax.plot(continuation_indexes, price_real[390:], color="C1", label="Real" if add_label else None)

    ax.plot(continuation_indexes, price_no_per[390:], color="C3", label="W/o perturbation" if add_label else None)

    # for i, (price_per_s, sigma_scaler) in enumerate(zip(price_per, SIGMA_SCALERs)):
    #     if i == 1:
    #         ax.plot(
    #             continuation_indexes,
    #             price_per_s[390:],
    #             color=f"C{i+4}",
    #             label=f"W/ perturbation: {sigma_scaler}" if add_label_ else None,
    #         )
    #         add_label_ = False

    ax.axvline(x=390, color="r")

    min_ = min(price_real.min(), price_no_per.min(), price_per.min())
    max_ = max(price_real.max(), price_no_per.max(), price_per.max())
    if stock_name == "NVDA":
        ax.add_patch(
            Rectangle(
                xy=(start_perturbation_n1, min_),
                width=start_perturbation_n2 - start_perturbation_n1,
                height=max_ - min_,
                color="r",
                alpha=0.3,
            )
        )
    ax.add_patch(
        Rectangle(
            xy=(start_perturbation_n2, min_),
            width=start_perturbation_n2 - start_perturbation_n1,
            height=max_ - min_,
            color="g",
            alpha=0.3,
        )
    )
    ax.add_patch(
        Rectangle(
            xy=(start_perturbation_n2 + 150, min_),
            width=price_no_per.shape[-1] - start_perturbation_n2,
            height=max_ - min_,
            color="b",
            alpha=0.1,
        )
    )
    add_label = False

    ax.set_title(stock_name, fontsize=FONT_SIZE_TITLE_AX)
    ax.set_xlabel("Steps", fontsize=FONT_SIZE_LABEL)
    ax.set_ylabel("Price ($)", fontsize=FONT_SIZE_LABEL, rotation=90)
    ax.xaxis.set_tick_params(labelsize=12)
    ax.yaxis.set_tick_params(labelsize=FONT_SIZE_TICKS)
    ax.set_xticks([0, 390, start_perturbation_n2, price_no_per.shape[-1]])

fig.suptitle("Introducing perturbations", fontsize=FONT_SIZE_TITLE_PLOT, y=1.0)
fig.legend(loc="upper center", ncol=3, fontsize=FONT_SIZE_LEGEND, frameon=False, bbox_to_anchor=(0.5, 0.97))
fig.tight_layout()
# plt.savefig(f"{PROJECT_FOLDER}/plot_finali/multistock/perturbations/perturbed_NVDA_one.pdf")
plt.show()
plt.close(fig)

In [ ]:
corr_real = np.round(np.corrcoef(prices_reals), 2)
corr_real

In [ ]:
corr_syntheticWoPer = np.round(np.corrcoef(priceWoPer[:, start_perturbation_n2 + 150 :]), 2)
corr_syntheticWoPer

In [ ]:
corr_syntheticWPer = np.round(np.corrcoef(priceWPer[:, 0, start_perturbation_n2 + 150 :]), 2)
corr_syntheticWPer

In [ ]:
priceWPer_KO, priceWPer_PEP, priceWPer_NVDA = priceWPer[:, 1, start_perturbation_n2 + 150 :]
priceWoPer_KO, priceWoPer_PEP, priceWoPer_NVDA = priceWoPer[:, start_perturbation_n2 + 150 :]

corr_KOWPer_PEPWoPer = round(np.corrcoef(priceWPer_KO, priceWoPer_PEP)[0, 1], 2)
corr_KOWPer_PEPWPer = round(np.corrcoef(priceWPer_KO, priceWPer_PEP)[0, 1], 2)

corr_KOWPer_NVDAWoPer = round(np.corrcoef(priceWPer_KO, priceWoPer_NVDA)[0, 1], 2)
corr_KOWPer_NVDAWPer = round(np.corrcoef(priceWPer_KO, priceWPer_NVDA)[0, 1], 2)

corr_PEPWPer_NVDAWoPer = round(np.corrcoef(priceWPer_PEP, priceWoPer_NVDA)[0, 1], 2)
corr_PEPWPer_NVDAWPer = round(np.corrcoef(priceWPer_PEP, priceWPer_NVDA)[0, 1], 2)

print(corr_KOWPer_PEPWoPer, corr_KOWPer_PEPWPer)
print(corr_KOWPer_NVDAWoPer, corr_KOWPer_NVDAWPer)
print(corr_PEPWPer_NVDAWoPer, corr_PEPWPer_NVDAWPer)

## Long perturbations

In [ ]:
seed = 1

In [ ]:
PATH_PICKLE_PRICE_NOPER = (
    f"{PROJECT_FOLDER}/storage/thesis-gan/{RUN_ID_PRICE}/perturbations/perturbation_seed={seed}_sigmascaler=None.pickle"
)

with open(PATH_PICKLE_PRICE_NOPER, "rb") as handle:
    dict_no_per = pickle.load(handle)

pricesWoPer = dict_no_per["pred_prices"].numpy()
pricesWoPer.shape

In [ ]:
PATH_PICKLE_PRICE_PER = (
    f"{PROJECT_FOLDER}/storage/thesis-gan/{RUN_ID_PRICE}/perturbations/"
    f"perturbation_seed={seed}_sigmascaler=0.1.pickle"
)

with open(PATH_PICKLE_PRICE_PER, "rb") as handle:
    dict_per = pickle.load(handle)

pricesWPer = dict_per["pred_prices"].numpy()
pricesWPer.shape

In [ ]:
history_indexes = np.arange(390)
continuation_indexes = np.arange(390, pricesWPer.shape[-1])
continuation_indexes_real = np.arange(390, prices_reals.shape[-1])

In [ ]:
i_start, i_end = 3, 15
start_perturbation, end_perturbation = 390 + i_start * 150, 390 + (i_end + 1) * 150
start_perturbation, end_perturbation

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 9))
axes = axes.ravel()

add_label = True
for (
    ax,
    stock_name,
    price_real,
    price_no_per,
    price_per,
) in zip(axes, stock_names, prices_reals, pricesWoPer, pricesWPer):
    ax.plot(history_indexes, price_real[:390], color="C0")
    ax.plot(continuation_indexes_real, price_real[390:], color="C1", label="Real" if add_label else None)

    ax.plot(continuation_indexes, price_no_per[390:], color="C3", label="W/o perturbation" if add_label else None)
    ax.plot(continuation_indexes, price_per[390:], color="C4", label="W/ perturbation" if add_label else None)

    ax.axvline(x=390, color="r")

    min_ = min(price_real.min(), price_no_per.min(), price_per.min())
    max_ = max(price_real.max(), price_no_per.max(), price_per.max())
    ax.add_patch(
        Rectangle(
            xy=(start_perturbation, min_),
            width=end_perturbation - start_perturbation,
            height=max_ - min_,
            color="r",
            alpha=0.3,
        )
    )
    add_label = False

    ax.set_title(stock_name, fontsize=FONT_SIZE_TITLE_AX)
    ax.set_xlabel("Steps", fontsize=FONT_SIZE_LABEL)
    ax.set_ylabel("Price ($)", fontsize=FONT_SIZE_LABEL, rotation=90)
    ax.xaxis.set_tick_params(labelsize=12)
    ax.yaxis.set_tick_params(labelsize=FONT_SIZE_TICKS)
    ax.set_xticks([0, 390, start_perturbation, end_perturbation])

fig.suptitle("Prices", fontsize=FONT_SIZE_TITLE_PLOT, y=1.0)
fig.legend(loc="upper center", ncol=3, fontsize=FONT_SIZE_LEGEND, frameon=False, bbox_to_anchor=(0.5, 0.97))
fig.tight_layout()
# plt.savefig(f"{PROJECT_FOLDER}/plot_finali/multistock/perturbations/long_perturbed.pdf")
plt.show()
plt.close(fig)

In [ ]:
corr_after_perturbation = round(np.corrcoef(pricesWPer[:, end_perturbation:])[0, 1], 2)
corr_during_perturbation = round(np.corrcoef(pricesWPer[:, start_perturbation:end_perturbation])[0, 1], 2)

In [ ]:
print(f"Real: {corr_real}")
print(f"After perturbation: {corr_after_perturbation}")
print(f"During perturbation: {corr_during_perturbation}")